# Import Libraries And Data

In [ ]:
import os
import pandas as pd
import platform
import numpy as np
from itertools import product
import matplotlib.pyplot as plt
from datetime import timedelta
from collections import defaultdict
import joblib
import json
import warnings
import time
from scipy.stats.mstats import winsorize
import numba
import pandas_ta as pta

import sys
import os

# Add one directory up to sys.path
sys.path.append(os.path.abspath(".."))

# Internal Libraries
from data_loader import load_and_resample_data, apply_feature_engineering
from backtest import evaluate_regression
from labeling_utils import label_and_save
from helpers import check_overfit, generate_oof_predictions, is_same_session
#

# Tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Dropout, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.preprocessing import StandardScaler, RobustScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import Huber, MeanSquaredError
import tensorflow as tf
#

# Scikit-learn
from sklearn.base import clone, BaseEstimator, RegressorMixin, ClassifierMixin
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, cross_val_predict
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression, RidgeClassifier, ElasticNet, Ridge
from sklearn.metrics import classification_report, root_mean_squared_error, mean_squared_error, mean_absolute_error, r2_score, \
    confusion_matrix, precision_recall_curve, roc_curve, auc, accuracy_score, classification_report, f1_score, precision_score, ConfusionMatrixDisplay
from sklearn.preprocessing import label_binarize, StandardScaler, LabelEncoder
from sklearn.inspection import permutation_importance
from sklearn.pipeline import make_pipeline
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight

# Models and Training
from catboost import CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb
import optuna
import seaborn as sns
import shap
from sklearn.svm import SVC
#

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", message=".*There are no meaningful features.*", category=UserWarning)
optuna.logging.set_verbosity(optuna.logging.INFO)

# Initialize features

In [ ]:
# This is your Cell 5: Feature Engineering Function Definitions

# --- Helper Functions for Custom Features (Some will be kept, some modified/called differently) ---
# These helpers now expect columns to be named as pandas_ta typically names them (e.g., RSI_14, EMA_10, MACD_12_26_9, etc.)

def add_price_vs_ma(df, price_col='close', ma_col_name='EMA_20', new_col_name_suffix='_vs_EMA20'):
    # Ensure ma_col_name exists (it would have been created by pandas_ta)
    if ma_col_name in df.columns and price_col in df.columns:
        # Ensure inputs are numeric before division
        df[price_col + new_col_name_suffix] = pd.to_numeric(df[price_col], errors='coerce') / pd.to_numeric(df[ma_col_name], errors='coerce')
    return df

def add_ma_vs_ma(df, ma1_col_name='EMA_10', ma2_col_name='EMA_20', new_col_name_suffix='_vs_EMA20'):
    if ma1_col_name in df.columns and ma2_col_name in df.columns:
        df[ma1_col_name + new_col_name_suffix] = pd.to_numeric(df[ma1_col_name], errors='coerce') / pd.to_numeric(df[ma2_col_name], errors='coerce')
    return df

def add_ma_slope(df, ma_col_name='EMA_10', new_col_name_suffix='_Slope_10', periods=1):
    if ma_col_name in df.columns:
        df[new_col_name_suffix] = pd.to_numeric(df[ma_col_name], errors='coerce').diff(periods) / periods
    return df

def add_rsi_signals(df, rsi_col_name='RSI_14', ob_level=70, os_level=30):
    if rsi_col_name in df.columns:
        rsi_series = pd.to_numeric(df[rsi_col_name], errors='coerce')
        df[rsi_col_name + f'_Is_Overbought_{ob_level}'] = (rsi_series > ob_level).astype(int)
        df[rsi_col_name + f'_Is_Oversold_{os_level}'] = (rsi_series < os_level).astype(int)
    return df

def add_stoch_signals(df, stoch_k_col_name='STOCHk_14_3_3', ob_level=80, os_level=20): # Default pandas_ta name for k
    if stoch_k_col_name in df.columns:
        stoch_k_series = pd.to_numeric(df[stoch_k_col_name], errors='coerce')
        df[stoch_k_col_name + f'_Is_Overbought_{ob_level}'] = (stoch_k_series > ob_level).astype(int)
        df[stoch_k_col_name + f'_Is_Oversold_{os_level}'] = (stoch_k_series < os_level).astype(int)
    return df

def add_macd_cross_signal(df, macd_col_name='MACD_12_26_9', signal_col_name='MACDs_12_26_9'): # Default pandas_ta name for signal
    if macd_col_name in df.columns and signal_col_name in df.columns:
        macd_series = pd.to_numeric(df[macd_col_name], errors='coerce')
        signal_series = pd.to_numeric(df[signal_col_name], errors='coerce')
        crossed_above = (macd_series > signal_series) & (macd_series.shift(1) < signal_series.shift(1))
        crossed_below = (macd_series < signal_series) & (macd_series.shift(1) > signal_series.shift(1))
        df[macd_col_name + '_Cross_Signal'] = np.where(crossed_above, 1, np.where(crossed_below, -1, 0))
    return df

def add_price_vs_bb(df, price_col='close', bb_upper_col='BBU_20_2.0', bb_lower_col='BBL_20_2.0'): # Default pandas_ta names
    if price_col in df.columns and bb_upper_col in df.columns and bb_lower_col in df.columns:
        price_series = pd.to_numeric(df[price_col], errors='coerce')
        bb_upper_series = pd.to_numeric(df[bb_upper_col], errors='coerce')
        bb_lower_series = pd.to_numeric(df[bb_lower_col], errors='coerce')
        df[price_col + '_vs_BB_Upper'] = (price_series > bb_upper_series).astype(int)
        df[price_col + '_vs_BB_Lower'] = (price_series < bb_lower_series).astype(int)
    return df

def add_psar_flip_signal(df, psar_col_name='PSARr_0.02_0.2', close_col='close'): # pandas_ta PSAR reversal column
    if psar_col_name in df.columns: # PSARr is 1 for reversal to uptrend, -1 for reversal to downtrend
        df['PSAR_Flip_Signal'] = pd.to_numeric(df[psar_col_name], errors='coerce').fillna(0).astype(int)
    return df

# Keep these custom functions as they are generally good:
def add_daily_vwap(df, high_col='high', low_col='low', close_col='close', volume_col='volume', new_col_name='VWAP_D'): # Changed name to VWAP_D for daily
    # ... (your existing robust add_daily_vwap function - ensure it uses .copy() and numeric conversions internally)
    # Make sure the final column is named VWAP_D or adjust add_price_vs_ma call later
    if not isinstance(df.index, pd.DatetimeIndex):
        print("Error: DataFrame index must be DatetimeIndex for daily VWAP.")
        return df
    df_temp = df.copy()
    for col in [high_col, low_col, close_col, volume_col]:
        df_temp[col] = pd.to_numeric(df_temp[col], errors='coerce')
    tpv = ((df_temp[high_col] + df_temp[low_col] + df_temp[close_col]) / 3) * df_temp[volume_col]
    cumulative_tpv = tpv.groupby(df_temp.index.date).cumsum()
    cumulative_volume = df_temp[volume_col].groupby(df_temp.index.date).cumsum()
    vwap_series = cumulative_tpv / cumulative_volume
    df[new_col_name] = vwap_series.replace([np.inf, -np.inf], np.nan)
    return df


def add_candle_features(df):
    # ... (your existing add_candle_features function - ensure numeric conversions) ...
    df_temp = df.copy()
    for col in ['open', 'high', 'low', 'close']:
        df_temp[col] = pd.to_numeric(df_temp[col], errors='coerce')
    df['Candle_Range'] = df_temp['high'] - df_temp['low']
    df['Candle_Body'] = (df_temp['close'] - df_temp['open']).abs()
    df['Upper_Wick'] = df_temp['high'] - np.maximum(df_temp['open'], df_temp['close'])
    df['Lower_Wick'] = np.minimum(df_temp['open'], df_temp['close']) - df_temp['low']
    df['Body_vs_Range'] = (df['Candle_Body'] / df['Candle_Range'].replace(0, np.nan)).replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

def add_return_features(df, price_col='close'):
    # ... (your existing add_return_features function - ensure numeric conversions and inf handling) ...
    price_series_num = pd.to_numeric(df[price_col], errors='coerce').replace(0, np.nan)
    df[f'Log_Return_1'] = np.log(price_series_num / price_series_num.shift(1))
    df[f'Log_Return_3'] = np.log(price_series_num / price_series_num.shift(3))
    df[f'Log_Return_6'] = np.log(price_series_num / price_series_num.shift(6))
    df[f'Simple_Return_1'] = price_series_num.pct_change(1)
    for col_ret in [f'Log_Return_1', f'Log_Return_3', f'Log_Return_6', f'Simple_Return_1']:
        if col_ret in df.columns: df[col_ret] = df[col_ret].replace([np.inf, -np.inf], np.nan)
    return df

def add_rolling_stats(df, price_col='close', window1=14, window2=30):
    # ... (your existing add_rolling_stats function - ensure numeric conversions and inf handling) ...
    returns = pd.to_numeric(df[price_col], errors='coerce').pct_change(1).replace([np.inf, -np.inf], np.nan)
    df[f'Rolling_Std_Dev_{window1}'] = returns.rolling(window=window1).std()
    df[f'Rolling_Skew_{window2}'] = returns.rolling(window=window2).skew()
    df[f'Rolling_Kurtosis_{window2}'] = returns.rolling(window=window2).kurt()
    return df

def add_lagged_features(df, cols_to_lag, lags=[1, 3, 6]):
    # ... (your existing add_lagged_features function - ensure numeric conversions on source col if needed) ...
    for col_orig in cols_to_lag:
        if col_orig in df.columns:
            series_to_lag = pd.to_numeric(df[col_orig], errors='coerce')
            for lag in lags:
                df[f'{col_orig}_Lag_{lag}'] = series_to_lag.shift(lag)
    return df


# --- Main Feature Generation Function using pandas_ta ---
def add_all_features(df_input, suffix=''):
    """
    Adds technical indicators and derived features using pandas_ta.
    Assumes df_input has 'open', 'high', 'low', 'close', 'volume' columns (DatetimeIndex).
    """
    if not isinstance(df_input.index, pd.DatetimeIndex):
        print(f"Warning: DataFrame for suffix '{suffix}' does not have a DatetimeIndex.")
    
    df = df_input.copy() # Work on a copy

    # Ensure base OHLCV columns are numeric and present
    base_cols = ['open', 'high', 'low', 'close', 'volume']
    if not all(col in df.columns for col in base_cols):
         raise ValueError(f"DataFrame must contain {base_cols}. Found: {df.columns.tolist()}")
    for col in base_cols: # Ensure correct dtypes for pandas_ta
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df.dropna(subset=base_cols, inplace=True) # Drop rows if OHLCV became NaN

    if df.empty:
        print(f"DataFrame became empty after coercing OHLCV for suffix '{suffix}'. Returning empty DataFrame.")
        # Return an empty dataframe with expected suffixed columns if possible or raise error
        # For simplicity, we'll let it create columns that will be all NaN, then suffixing will apply.
        # Or handle more gracefully by creating expected columns with NaNs.
        # For now, we will proceed, and suffixing will apply to what gets created.
        pass


    print(f"DataFrame shape for pandas_ta (suffix: {suffix}): {df.shape}")

    # I. Technical Indicators using pandas_ta
    # Most pandas_ta functions automatically name columns (e.g., SMA_10, RSI_14)
    # and handle NaNs internally. `append=True` adds them to df.

    # Volume
    df.ta.sma(close=df['volume'], length=20, append=True, col_names=('Volume_SMA_20'))
    df = add_daily_vwap(df, new_col_name='VWAP_D') # Using your custom daily VWAP, named VWAP_D
    df = add_price_vs_ma(df, ma_col_name='VWAP_D', new_col_name_suffix='_vs_VWAP_D')

    # Volatility
    df.ta.bbands(length=20, std=2, append=True) # Creates BBL_20_2.0, BBM_20_2.0, BBU_20_2.0, BBB_20_2.0, BBP_20_2.0
    # Helpers will need these names: BBU_20_2.0, BBL_20_2.0
    df = add_price_vs_bb(df, bb_upper_col='BBU_20_2.0', bb_lower_col='BBL_20_2.0')
    df.ta.atr(length=14, append=True, col_names=('ATR_14')) # pandas_ta might name it ATRr_14 or similar. We force ATR_14.

    # Trend
    df.ta.sma(length=10, append=True) # SMA_10
    df.ta.sma(length=20, append=True) # SMA_20 (also BBM_20_2.0 from bbands)
    df.ta.sma(length=50, append=True) # SMA_50
    df.ta.ema(length=10, append=True) # EMA_10
    df.ta.ema(length=20, append=True) # EMA_20
    df.ta.ema(length=50, append=True) # EMA_50
    
    df = add_price_vs_ma(df, ma_col_name='EMA_20', new_col_name_suffix='_vs_EMA20')
    df = add_ma_vs_ma(df, ma1_col_name='EMA_10', ma2_col_name='EMA_20', new_col_name_suffix='_vs_EMA20')
    df = add_ma_slope(df, ma_col_name='EMA_10', new_col_name_suffix='_Slope_10')

    df.ta.macd(fast=12, slow=26, signal=9, append=True) # MACD_12_26_9, MACDh_12_26_9, MACDs_12_26_9
    df = add_macd_cross_signal(df, macd_col_name='MACD_12_26_9', signal_col_name='MACDs_12_26_9')

    df.ta.adx(length=14, append=True) # ADX_14, DMP_14, DMN_14
    # Rename DMN_14 and DMP_14 to match your old Minus_DI_14, Plus_DI_14 if helpers depend on it
    if 'DMP_14' in df.columns: df.rename(columns={'DMP_14': 'Plus_DI_14'}, inplace=True)
    if 'DMN_14' in df.columns: df.rename(columns={'DMN_14': 'Minus_DI_14'}, inplace=True)
    
    df.ta.psar(append=True) # Creates PSARl_0.02_0.2, PSARs_0.02_0.2, PSARaf_0.02_0.2, PSARr_0.02_0.2
    df = add_psar_flip_signal(df, psar_col_name='PSARr_0.02_0.2') # Use reversal column

    df.ta.cci(length=20, append=True, col_names=('CCI_20')) # pandas_ta uses CCI_20_0.015 by default

    # Momentum
    df.ta.rsi(length=14, append=True) # RSI_14
    df = add_rsi_signals(df, rsi_col_name='RSI_14')
    
    df.ta.stoch(k=14, d=3, smooth_k=3, append=True) # STOCHk_14_3_3, STOCHd_14_3_3
    df = add_stoch_signals(df, stoch_k_col_name='STOCHk_14_3_3')

    df.ta.ppo(fast=12, slow=26, signal=9, append=True) # PPO_12_26_9, PPOh_12_26_9, PPOs_12_26_9
    df.ta.roc(length=10, append=True) # ROC_10
    
    # Explicitly convert PPO and ROC to numeric (belt and braces after pandas_ta)
    for col_name in ['PPO_12_26_9', 'ROC_10']: # Check exact names if pandas_ta produces variants
        base_name_ppo = [c for c in df.columns if "PPO_" in c and "PPOh" not in c and "PPOs" not in c]
        base_name_roc = [c for c in df.columns if "ROC_" in c]
        
        for actual_col_name in base_name_ppo + base_name_roc:
            if actual_col_name in df.columns:
                df[actual_col_name] = df[actual_col_name].replace([np.inf, -np.inf], np.nan)
                df[actual_col_name] = pd.to_numeric(df[actual_col_name], errors='coerce')

    # II. Price Action & Basic Features (Keep your custom functions)
    df = add_candle_features(df)
    # df = add_candlestick_patterns(df) # We'll replace this with pandas_ta candlestick patterns

    # --- pandas_ta Candlestick Patterns ---
    # Example: Add Doji, Hammer, Engulfing. pandas_ta has many more.
    # 'name="all"' would add many columns, so be selective or use a list.
    candle_patterns_to_check = ["doji", "hammer", "engulfing"] 
    df.ta.cdl_pattern(name=candle_patterns_to_check, append=True)
    # Rename columns to match your old convention if needed, e.g., CDLDOJI -> Is_Doji
    if 'CDLDOJI' in df.columns: df.rename(columns={'CDLDOJI': 'Is_Doji_pta'}, inplace=True) # Add _pta to distinguish
    if 'CDLHAMMER' in df.columns: df.rename(columns={'CDLHAMMER': 'Is_Hammer_pta'}, inplace=True)
    if 'CDLENGULFING' in df.columns: df.rename(columns={'CDLENGULFING': 'Is_Engulfing_pta'}, inplace=True) # This is a general engulfing signal (+/-)

    df = add_return_features(df)

    # III. Statistical Features (Keep your custom functions)
    df = add_rolling_stats(df)
    
    # Lagged Features
    # Ensure base columns for lagging are the ones created by pandas_ta or your helpers
    cols_to_lag_pta = ['close', 'RSI_14', 'Candle_Body', 'Volume_SMA_20'] 
    # Check if these columns actually exist, as pandas_ta might name them slightly differently
    # This valid_cols_to_lag should use the names as they are in df at this point
    valid_cols_to_lag = [col for col in cols_to_lag_pta if col in df.columns]
    df = add_lagged_features(df, valid_cols_to_lag, lags=[1,2,3])

    # --- Suffixing ---
    # All columns created by pandas_ta (that were appended) or by helpers
    # that are not the original 'open', 'high', 'low', 'close', 'volume' will be suffixed.
    current_cols = list(df.columns)
    # Identify features generated in this function call (not the original base OHLCV)
    generated_feature_cols = [col for col in current_cols if col not in base_cols]
    
    rename_dict = {col: col + suffix for col in generated_feature_cols}
    df.rename(columns=rename_dict, inplace=True)
    
    return df

# --- Time & Session Features (Keep as is) ---
def add_time_session_features(df):
    # ... (your existing add_time_session_features function) ...
    if not isinstance(df.index, pd.DatetimeIndex):
        print("Error: DataFrame index must be DatetimeIndex for time/session features.")
        return df
    df = df.copy()
    df['Hour_of_Day'] = df.index.hour
    df['Minute_of_Hour'] = df.index.minute
    df['Day_of_Week'] = df.index.dayofweek
    time_fraction_of_day = df['Hour_of_Day'] + df['Minute_of_Hour'] / 60.0
    df['Time_Sin'] = np.sin(2 * np.pi * time_fraction_of_day / 24.0)
    df['Time_Cos'] = np.cos(2 * np.pi * time_fraction_of_day / 24.0)
    df['Day_Sin'] = np.sin(2 * np.pi * df['Day_of_Week'] / 7.0)
    df['Day_Cos'] = np.cos(2 * np.pi * df['Day_of_Week'] / 7.0)
    df['Is_Asian_Session'] = ((df['Hour_of_Day'] >= 20) | (df['Hour_of_Day'] < 5)).astype(int)
    df['Is_London_Session'] = ((df['Hour_of_Day'] >= 3) & (df['Hour_of_Day'] < 12)).astype(int)
    df['Is_NY_Session'] = ((df['Hour_of_Day'] >= 8) & (df['Hour_of_Day'] < 17)).astype(int)
    df['Is_Overlap'] = ((df['Hour_of_Day'] >= 8) & (df['Hour_of_Day'] < 12)).astype(int)
    df['Is_US_Open_Hour'] = ((df['Hour_of_Day'] == 9) & (df['Minute_of_Hour'] >= 30) | (df['Hour_of_Day'] == 10) & (df['Minute_of_Hour'] < 30)).astype(int)
    df['Is_US_Close_Hour'] = ((df['Hour_of_Day'] == 15) | (df['Hour_of_Day'] == 16) & (df['Minute_of_Hour'] == 0)).astype(int)
    return df

In [ ]:
param_grid_strategy = {
    'SL_ATR_MULT': [1.0, 1.5, 0.5],
    'TP_ATR_MULT': [2.0, 3.0, 4.0],
    'TRAIL_START_MULT': [0.5, 1.0],
    'TRAIL_STOP_MULT': [0.5, 1.0],
    'TICK_VALUE': [5], 
}

keys, values = zip(*param_grid_strategy.items())
combinations = [dict(zip(keys, v)) for v in product(*values)]

avoid_funcs = {
    #'avoid_hour_18_19': avoid_hour_18_19
    #'news_window': avoid_news,
}

# Cleanup

In [ ]:
df_1min, resampled = load_and_resample_data(timeframes=['5min', '15min','1h'])

df_5min = resampled['5min']
df_15min = resampled['15min']
df_1hr = resampled['1h']

print("5min shape:", df_5min.shape)
print("15min shape:", df_15min.shape)
print("1h shape:", df_1hr.shape)


df_merged = apply_feature_engineering(
    resampled=resampled,
    add_all_features=add_all_features,
    add_time_session_features=add_time_session_features,
    timeframes=['5min', '15min', '1h'],
    base_tf='5min'
)

# Verify Data
# def check_feature_alignment(df_merged, tf_suffix, feature_keyword, sample_times):
#     """
#     Check if features from the correct timeframe are available and what values they have at key timestamps.
#     """
#     suffix_str = f"_{tf_suffix}"
#     matching_cols = [col for col in df_merged.columns if feature_keyword in col and col.endswith(suffix_str)]
    
#     print(f"\n🔍 Checking features containing '{feature_keyword}' with suffix '{suffix_str}'")

#     if not matching_cols:
#         print(f"⚠️ No matching columns found. Available columns ending with {suffix_str}:")
#         sample_suffix_cols = [col for col in df_merged.columns if col.endswith(suffix_str)]
#         print(sample_suffix_cols[:10])  # Show only first 10 to keep it clean
#         return

#     for time in sample_times:
#         if time not in df_merged.index:
#             print(f"❌ Time {time} not found in df_merged index.")
#         else:
#             print(f"\n⏰ At time {time} — values:")
#             print(df_merged.loc[time, matching_cols])

# sample_times = [
#     pd.Timestamp("2022-01-05 12:05:00-05:00"),
#     pd.Timestamp("2022-01-05 12:15:00-05:00"),
#     pd.Timestamp("2022-01-05 12:59:00-05:00"),
#     pd.Timestamp("2022-01-05 13:00:00-05:00")
# ]

# check_feature_alignment(df_merged, tf_suffix="1h", feature_keyword="RSI", sample_times=sample_times)
# check_feature_alignment(df_merged, tf_suffix="15min", feature_keyword="MACD", sample_times=sample_times)
# check_feature_alignment(df_merged, tf_suffix="5min", feature_keyword="EMA", sample_times=sample_times)


In [ ]:
labeled = label_and_save(
    df_input_features=df_merged,
    lookahead_period=12,
    vol_col_name='ATR_14_5min',
    pt_multiplier=2.0,
    sl_multiplier=1.0,
    min_return_percentage=0.0005,
    output_file_suffix='L12_PT2SL1VB12-model-research',
    feature_columns_for_dropna=[]
)

# Train

##### Wrappers

In [ ]:
class LSTMWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, input_shape, units=32, dropout=0.2, lr=0.001, epochs=10, batch_size=32, verbose=0):
        self.input_shape = input_shape
        self.units = units
        self.dropout = dropout
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = None

    def build_model(self):
        model = Sequential()
        model.add(LSTM(self.units, input_shape=(self.input_shape, 1)))
        model.add(Dropout(self.dropout))
        model.add(Dense(1))
        model.compile(optimizer=Adam(learning_rate=self.lr), loss='mse')
        return model

    def fit(self, X, y):
        print(f"🧠 [ImprovedLSTMWrapper] Starting training with {X.shape[0]} samples, {X.shape[1]} features")
        X = X.reshape((X.shape[0], X.shape[1], 1))
        self.model = self.build_model()
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        print(f"✅ [LSTMWrapper] Finished training.")
        return self

    def predict(self, X):
        X = X.reshape((X.shape[0], X.shape[1], 1))
        return self.model.predict(X).flatten()

In [ ]:
class CNN1DWrapper:
    def __init__(self, input_shape):
        self.input_shape = input_shape
        self.model = self.build_model()
        self.scaler = StandardScaler()

    def build_model(self):
        model = Sequential()
        model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=self.input_shape))
        model.add(Conv1D(32, kernel_size=3, activation='relu'))
        model.add(GlobalAveragePooling1D())
        model.add(Dropout(0.3))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(1, activation='tanh'))  # outputs in [-1, 1]
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')
        return model

    def fit(self, X, y, epochs=20, batch_size=128, verbose=1):
        print(f"\n🔧 [CNN1DWrapper] Scaling target and starting training...")
        y_scaled = self.scaler.fit_transform(y.reshape(-1, 1)).ravel()
        start = time.time()
        self.model.fit(X, y_scaled, epochs=epochs, batch_size=batch_size, verbose=verbose)
        print(f"✅ [CNN1DWrapper] Training complete in {time.time() - start:.2f} seconds.")

    def predict(self, X):
        print(f"\n🔮 [CNN1DWrapper] Predicting on {X.shape}...")
        start = time.time()
        y_scaled_pred = self.model.predict(X).ravel()
        y_pred = self.scaler.inverse_transform(y_scaled_pred.reshape(-1, 1)).ravel()
        print(f"✅ [CNN1DWrapper] Prediction done in {time.time() - start:.2f} seconds.")

        print("🔍 Prediction Stats:")
        print(f"Min: {y_pred.min():.6f} | Max: {y_pred.max():.6f}")
        print(f"Mean: {y_pred.mean():.6f} | Std: {y_pred.std():.6f}")

        # Safety check for wild predictions
        if abs(y_pred).max() > 1:
            print("⚠️ Warning: Some predictions exceed ±1 — consider checking target scaling or model output activation.")

        return y_pred

In [ ]:
class LSTMClassifierWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, input_shape, num_classes=5, epochs=10, batch_size=32, verbose=0):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = None

    def build_model(self):
        model = Sequential()
        model.add(LSTM(32, input_shape=(self.input_shape, 1)))  # fixed 32 units
        model.add(Dropout(0.2))
        model.add(Dense(self.num_classes, activation="softmax"))
        model.compile(
            loss="categorical_crossentropy",
            optimizer=Adam(learning_rate=0.001),
            metrics=["accuracy"]
        )
        return model

    def fit(self, X, y):
        print(f"🧠 [LSTMClassifierWrapper] Training on {X.shape[0]} samples, {X.shape[1]} features")
        X = np.array(X).reshape((len(X), self.input_shape, 1))
        self.model = self.build_model()

        # If multiclass and y is integer labels, one-hot encode
        if self.num_classes > 2 and y.ndim == 1:
            y = to_categorical(y, num_classes=self.num_classes)

        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        print(f"✅ [LSTMClassifierWrapper] Training complete.")
        return self

    def predict(self, X):
        X = np.array(X).reshape((len(X), self.input_shape, 1))
        probs = self.model.predict(X)
        return np.argmax(probs, axis=1)

    def predict_proba(self, X):
        X = np.array(X).reshape((len(X), self.input_shape, 1))
        return self.model.predict(X)

##### Model Tunning

In [ ]:
# === 🔧 Model Optimizers (XGBoost, CatBoost, RF, LightGBM)  CLASSIFICATION ===
def tune_xgboost(
    X_train, y_train, splits, trials, num_classes,
    study_name="xgboost_opt", db_path="sqlite:///optuna_studies/xgb_opt.db"
):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0.0, 5.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'objective': 'multi:softmax',
            'eval_metric': 'mlogloss',
            'num_class': num_classes,
        }

        tscv = TimeSeriesSplit(n_splits=splits)
        scores = []

        for train_idx, val_idx in tscv.split(X_train):
            X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_tr, y_val = y_train[train_idx], y_train[val_idx]

            model = xgb.XGBClassifier(**params, n_jobs=-4)
            model.fit(X_tr, y_tr)
            preds = model.predict(X_val)
            scores.append(f1_score(y_val, preds, average="macro"))

        return np.mean(scores)

    study = optuna.create_study(
        direction="maximize",
        study_name=study_name,
        storage=db_path,
        load_if_exists=True
    )
    study.optimize(objective, n_trials=trials)
    return study.best_trial.params

def tune_catboost(X, y, splits, trials, num_classes, study_name, db_path):
    def objective(trial):
        bootstrap_type = trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli'])

        class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)

        params = {
            'iterations': trial.suggest_int('iterations', 100, 1000, step=100),
            'depth': trial.suggest_int('depth', 4, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
            'bootstrap_type': bootstrap_type,
            'loss_function': 'MultiClass',
            'eval_metric': 'TotalF1',
            'class_weights': class_weights.tolist(),
            'verbose': 0
        }

        if bootstrap_type == 'Bayesian':
            params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 1)

        model = CatBoostClassifier(**params)
        tscv = TimeSeriesSplit(n_splits=splits)
        scores = cross_val_score(model, X, y, cv=tscv, scoring='f1_macro', n_jobs=-4)
        return scores.mean()

    study = optuna.create_study(direction="maximize", study_name=study_name, storage=db_path, load_if_exists=True)
    study.optimize(objective, n_trials=trials)
    return study.best_trial.params

def tune_rf(X, y, splits, trials, study_name, db_path):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'max_depth': trial.suggest_int('max_depth', 5, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
            'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced']),
        }

        model = RandomForestClassifier(**params, random_state=42)
        tscv = TimeSeriesSplit(n_splits=splits)
        scores = cross_val_score(model, X, y, cv=tscv, scoring="f1_macro", n_jobs=-4)
        return scores.mean()

    study = optuna.create_study(direction="maximize", study_name=study_name, storage=db_path, load_if_exists=True)
    study.optimize(objective, n_trials=trials)
    return study.best_trial.params

def tune_lightgbm(X, y, splits, trials, num_classes, study_name, db_path):
    def objective(trial):
        params = {
            'objective': 'multiclass',
            'num_class': num_classes,
            'metric': 'multi_logloss',
            'boosting_type': 'gbdt',
            'verbosity': -1,
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 31, 128),
            'max_depth': trial.suggest_int('max_depth', 5, 15),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 10)
        }

        scores = []
        tscv = TimeSeriesSplit(n_splits=splits)
        for train_idx, val_idx in tscv.split(X):
            model = lgb.LGBMClassifier(**params)
            model.fit(X.iloc[train_idx], y[train_idx])
            preds = model.predict(X.iloc[val_idx])
            scores.append(f1_score(y[val_idx], preds, average='macro'))

        return np.mean(scores)

    study = optuna.create_study(direction="maximize", study_name=study_name, storage=db_path, load_if_exists=True)
    study.optimize(objective, n_trials=trials)
    return study.best_trial.params

def tune_logistic_regression(X, y, splits, trials, study_name, db_path):
    def objective(trial):
        penalty = trial.suggest_categorical("penalty", ["l2", None])
        C = trial.suggest_float("C", 1e-4, 10.0, log=True)
        class_weight = trial.suggest_categorical("class_weight", [None, "balanced"])

        params = {
            "penalty": penalty,
            "C": C,
            "solver": "lbfgs",
            "multi_class": "multinomial",
            "max_iter": 2000,
            "class_weight": class_weight
        }

        model = make_pipeline(StandardScaler(), LogisticRegression(**params))
        tscv = TimeSeriesSplit(n_splits=splits)
        scores = cross_val_score(model, X, y, cv=tscv, scoring="f1_macro", n_jobs=-4)
        return scores.mean()

    study = optuna.create_study(direction="maximize", study_name=study_name, storage=db_path, load_if_exists=True)
    study.optimize(objective, n_trials=trials)
    return study.best_trial.params

def tune_ridge_classifier(X, y, splits, trials, study_name, db_path):
    def objective(trial):
        alpha = trial.suggest_float("alpha", 0.01, 10.0, log=True)
        solver = trial.suggest_categorical("solver", ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"])

        model = make_pipeline(
            StandardScaler(),
            RidgeClassifier(alpha=alpha, solver=solver, class_weight="balanced")
        )

        tscv = TimeSeriesSplit(n_splits=splits)
        scores = cross_val_score(model, X, y, cv=tscv, scoring="f1_macro", n_jobs=-4)
        return scores.mean()

    study = optuna.create_study(direction="maximize", study_name=study_name, storage=db_path, load_if_exists=True)
    study.optimize(objective, n_trials=trials)
    return study.best_trial.params

def tune_svc(X, y, splits, trials, study_name, db_path):
    def objective(trial):
        C = trial.suggest_float("C", 0.1, 10.0, log=True)
        kernel = trial.suggest_categorical("kernel", ["linear", "rbf", "poly"])
        gamma = trial.suggest_categorical("gamma", ["scale", "auto"])

        model = make_pipeline(
            StandardScaler(),
            SVC(C=C, kernel=kernel, gamma=gamma, class_weight='balanced')
        )

        tscv = TimeSeriesSplit(n_splits=splits)
        scores = cross_val_score(model, X, y, cv=tscv, scoring="f1_macro", n_jobs=-4)
        return scores.mean()

    study = optuna.create_study(direction="maximize", study_name=study_name, storage=db_path, load_if_exists=True)
    study.optimize(objective, n_trials=trials)
    return study.best_trial.params

In [ ]:
# === 🔧 Model Optimizers (XGBoost, CatBoost, RF, LightGBM)  Regression ===
def tune_xgb_regressor(X, y, splits, trials, study_name, db_path):
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "gamma": trial.suggest_float("gamma", 0.0, 5.0),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        }
        model = xgb.XGBRegressor(**params, n_jobs=-4)
        scores = cross_val_score(model, X, y, cv=TimeSeriesSplit(n_splits=splits), scoring="neg_mean_squared_error", n_jobs=-4)
        return scores.mean()
    study = optuna.create_study(direction="maximize", study_name=study_name, storage=db_path, load_if_exists=True)
    study.optimize(objective, n_trials=trials)
    return study.best_trial.params

# === LightGBM Regressor ===
def tune_lgb_regressor(X, y, splits, trials, study_name, db_path):
    def objective(trial):
        params = {
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 31, 128),
            "max_depth": trial.suggest_int("max_depth", 5, 15),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 10)
        }
        model = lgb.LGBMRegressor(**params)
        scores = cross_val_score(model, X, y, cv=TimeSeriesSplit(n_splits=splits), scoring="neg_mean_squared_error", n_jobs=-4)
        return scores.mean()
    study = optuna.create_study(direction="maximize", study_name=study_name, storage=db_path, load_if_exists=True)
    study.optimize(objective, n_trials=trials)
    return study.best_trial.params

# === CatBoost Regressor ===
def tune_catboost_regressor(X, y, splits, trials, study_name, db_path):
    def objective(trial):
        params = {
            "iterations": trial.suggest_int("iterations", 100, 1000, step=100),
            "depth": trial.suggest_int("depth", 4, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
            "random_strength": trial.suggest_float("random_strength", 0.5, 5.0),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        }
        model = CatBoostRegressor(**params, verbose=0)
        scores = cross_val_score(model, X, y, cv=TimeSeriesSplit(n_splits=splits), scoring="neg_mean_squared_error", n_jobs=-4)
        return scores.mean()
    study = optuna.create_study(direction="maximize", study_name=study_name, storage=db_path, load_if_exists=True)
    study.optimize(objective, n_trials=trials)
    return study.best_trial.params

# === Random Forest Regressor ===
def tune_rf_regressor(X, y, splits, trials, study_name, db_path):
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "max_depth": trial.suggest_int("max_depth", 5, 20),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        }
        model = RandomForestRegressor(**params, random_state=42)
        scores = cross_val_score(model, X, y, cv=TimeSeriesSplit(n_splits=splits), scoring="neg_mean_squared_error", n_jobs=-4)
        return scores.mean()
    study = optuna.create_study(direction="maximize", study_name=study_name, storage=db_path, load_if_exists=True)
    study.optimize(objective, n_trials=trials)
    return study.best_trial.params

# === Ridge Regressor ===
def tune_ridge_regressor(X, y, splits, trials, study_name, db_path):
    def objective(trial):
        alpha = trial.suggest_float("alpha", 0.01, 10.0, log=True)
        model = make_pipeline(StandardScaler(), Ridge(alpha=alpha))
        scores = cross_val_score(model, X, y, cv=TimeSeriesSplit(n_splits=splits), scoring="neg_mean_squared_error", n_jobs=-4)
        return scores.mean()
    study = optuna.create_study(direction="maximize", study_name=study_name, storage=db_path, load_if_exists=True)
    study.optimize(objective, n_trials=trials)
    return study.best_trial.params

# === ElasticNet Regressor ===
def tune_elasticnet_regressor(X, y, splits, trials, study_name, db_path):
    def objective(trial):
        alpha = trial.suggest_float("alpha", 1e-4, 10.0, log=True)
        l1_ratio = trial.suggest_float("l1_ratio", 0.0, 1.0)
        model = make_pipeline(StandardScaler(), ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=10000))
        scores = cross_val_score(model, X, y, cv=TimeSeriesSplit(n_splits=splits), scoring="neg_mean_squared_error", n_jobs=-4)
        return scores.mean()
    study = optuna.create_study(direction="maximize", study_name=study_name, storage=db_path, load_if_exists=True)
    study.optimize(objective, n_trials=trials)
    return study.best_trial.params

##### Real Training

In [ ]:
def analyze_classifier(name, model, X, y_true, feature_names=None):
    print(f"\n🔍 Evaluating {name}")
    preds = model.predict(X)
    probs = model.predict_proba(X) if hasattr(model, "predict_proba") else None

    acc = accuracy_score(y_true, preds)
    f1 = f1_score(y_true, preds, average='macro')
    prec = precision_score(y_true, preds, average='macro')

    print(f"✅ Accuracy: {acc:.4f}")
    print(f"🎯 Macro F1 Score: {f1:.4f}")
    print(f"📌 Precision (macro): {prec:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_true, preds))

    # Confusion Matrix
    cm = confusion_matrix(y_true, preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues')
    plt.title(f"{name} - Confusion Matrix")
    plt.show()

    # Feature Importance
    if hasattr(model, "feature_importances_"):
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1]
        top_features = [feature_names[i] for i in indices[:20]] if feature_names else None
        plt.figure(figsize=(10, 6))
        sns.barplot(x=importances[indices[:20]], y=top_features)
        plt.title(f"{name} - Top 20 Feature Importances")
        plt.xlabel("Importance")
        plt.show()

    elif hasattr(model, "coef_"):
        coef = model.coef_
        if coef.ndim == 2:
            coef = coef[0]
        if feature_names:
            plt.figure(figsize=(10, 6))
            sns.barplot(x=np.abs(coef), y=feature_names)
            plt.title(f"{name} - Coefficient Magnitudes")
            plt.xlabel("Magnitude")
            plt.show()

    return preds, probs

# === 🧠 Main training pipeline ===
def run_classification_pipeline(
    lookahead: int,
    parquet_path: str,
    feature_groups: dict,
    model_list: list,
    train_end: str,
    val_end: str,
    splits: int = 3,
    trials: int = 20,
    num_classes: int = 3,
    study_folder: str = "optuna_studies"
):
    """
    Train and evaluate multiple classifiers on labeled trading data.

    Parameters:
        lookahead: Lookahead window (used for study naming)
        parquet_path: Path to labeled parquet file
        feature_groups: Dict with keys ['tree', 'linear', 'sequential'] and feature name lists
        model_list: Models to run ['xgboost', 'catboost', 'rf', 'lightgbm', 'logreg']
        train_end: Timestamp (str) to end training data
        val_end: Timestamp (str) to end validation data
        splits: TimeSeriesSplit folds
        trials: Optuna trials
        num_classes: Number of label classes (usually 3 for triple-barrier)
        study_folder: Directory to store Optuna .db files
    """
    os.makedirs(study_folder, exist_ok=True)

    # === Load & split data ===
    df = pd.read_parquet(parquet_path)

    # 🔁 Recover datetime if it's the index
    if df.index.name == 'datetime' or pd.api.types.is_datetime64_any_dtype(df.index):
        df = df.reset_index()

    # ✅ Ensure datetime column exists
    if 'datetime' not in df.columns:
        raise KeyError("❌ 'datetime' column is missing. Make sure it exists or was saved as index.")

    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('datetime')

    train = df[df['datetime'] < train_end]
    val = df[(df['datetime'] >= train_end) & (df['datetime'] < val_end)]
    test = df[df['datetime'] >= val_end]  # Hold out for backtest

    print(f"Train rows: {len(train)} | Val rows: {len(val)} | Test rows: {len(test)}")

    # === Features & labels ===
    X_train = train[feature_groups['tree']]
    X_val = val[feature_groups['tree']]

    le = LabelEncoder()
    clf_col = [col for col in df.columns if col.startswith("clf_target")][0]
    print(f"📌 Using classification label column: {clf_col}")
    y_train = le.fit_transform(train[clf_col])
    y_val = le.transform(val[clf_col])

    results = {}

    # === Loop through models ===
    for model_name in model_list:
        print(f"\n=== 🔄 Running {model_name.upper()} training ===")

        study_name = f"{model_name}_class_opt_{lookahead}"
        db_path = f"sqlite:///{study_folder}/{study_name}.db"

        if model_name == "xgboost":
            import xgboost as xgb
            from xgboost import XGBClassifier
            def tuner(X, y): return tune_xgboost(X, y, splits, trials, num_classes, study_name, db_path)
            ModelClass = XGBClassifier

        elif model_name == "catboost":
            from catboost import CatBoostClassifier
            def tuner(X, y): return tune_catboost(X, y, splits, trials, num_classes, study_name, db_path)
            ModelClass = CatBoostClassifier

        elif model_name == "rf":
            from sklearn.ensemble import RandomForestClassifier
            def tuner(X, y): return tune_rf(X, y, splits, trials, study_name, db_path)
            ModelClass = RandomForestClassifier

        elif model_name == "lightgbm":
            import lightgbm as lgb
            def tuner(X, y): return tune_lightgbm(X, y, splits, trials, num_classes, study_name, db_path)
            ModelClass = lgb.LGBMClassifier

        elif model_name == "logreg":
            from sklearn.linear_model import LogisticRegression
            def tuner(X, y): return tune_logistic_regression(X, y, splits, trials, study_name, db_path)
            def ModelClass(**params): return make_pipeline(StandardScaler(), LogisticRegression(**params))

        else:
            print(f"⚠️ Unknown model: {model_name}")
            continue

        best_params = tuner(X_train, y_train)
        model = ModelClass(**best_params)
        model.fit(X_train, y_train)

        preds, probs = analyze_classifier(
            name=model_name.upper(),
            model=model,
            X=X_val,
            y_true=y_val,
            feature_names=X_train.columns.tolist()
        )

        results[model_name] = {
            "model": model,
            "params": best_params,
            "predictions": preds,
            "probabilities": probs
        }

    return results, le, df

In [ ]:
def analyze_regression(name, model, X, y_true, y_dates=None, entry_prices=None, exit_prices=None):
    preds = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y_true, preds))
    r2 = r2_score(y_true, preds)
    mae = mean_absolute_error(y_true, preds)

    print(f"\n📈 {name.upper()} REGRESSION RESULTS")
    print(f"✅ RMSE: {rmse:.6f}")
    print(f"🎯 R² Score: {r2:.4f}")
    print(f"📌 MAE: {mae:.6f}")

    # Residual plot
    residuals = y_true - preds
    plt.figure(figsize=(10, 4))
    plt.hist(residuals, bins=50, alpha=0.6, edgecolor='k')
    plt.title(f"{name.upper()} Residual Distribution")
    plt.xlabel("Residual (Actual - Predicted)")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

    # Directional accuracy
    accuracy = (np.sign(y_true) == np.sign(preds)).mean()
    print(f"📊 Directional Accuracy: {accuracy:.2%}")

    # Price-based PnL
    if y_dates is not None and entry_prices is not None and exit_prices is not None:
        pred_direction = np.sign(preds)
        valid = entry_prices.notna() & exit_prices.notna()

        entry_valid = entry_prices[valid].reset_index(drop=True)
        exit_valid = exit_prices[valid].reset_index(drop=True)
        dates_valid = y_dates[valid.values].reset_index(drop=True)
        price_return = (exit_valid - entry_valid) / entry_valid

        point_pnl = pred_direction[valid] * (exit_valid - entry_valid)
        dollar_pnl = point_pnl * 20  # NQ multiplier is $20/point

        pnl_series = pd.Series(dollar_pnl.values, index=dates_valid)
        pnl_series = pnl_series.cumsum()

        pnl_series.plot(title=f"{name.upper()} Realistic Price-Based PnL (NQ Futures)", figsize=(10, 4))
        plt.grid(True)
        plt.ylabel("Cumulative PnL ($)")
        plt.show()

    # SHAP interaction plot for tree-based models
    if name.lower() in ["xgboost", "lightgbm", "catboost", "rf"]:
        print(f"\n📊 SHAP Interaction Analysis for {name.upper()}")

        explainer = shap.TreeExplainer(model)
        interaction_values = explainer.shap_interaction_values(X)

        # Handle multiclass case (not expected in regression)
        if isinstance(interaction_values, list):
            interaction_values = interaction_values[0]

        feature_names = X.columns if hasattr(X, 'columns') else [f'feature_{i}' for i in range(X.shape[1])]

        # === Self-contribution (diagonal of mean interaction matrix) ===
        diag_matrix = np.diagonal(interaction_values, axis1=1, axis2=2)
        diag_mean = np.abs(diag_matrix).mean(axis=0)
        feature_self_impact = pd.Series(diag_mean, index=feature_names).sort_values()

        print("\n🔻 Features with Least Self-Impact (may not help alone):")
        print(feature_self_impact.head(15).to_string())

        # === Total contribution (sum of all interactions per feature) ===
        total_contrib = np.abs(interaction_values).sum(axis=(0, 1))
        feature_total_impact = pd.Series(total_contrib, index=feature_names).sort_values()

        print("\n❌ Features with Lowest Total Contribution (including interactions):")
        print(feature_total_impact.head(15).to_string())

        # === 🧹 Suggest dropping features with total impact == 0
        zero_impact_features = feature_total_impact[feature_total_impact == 0].index.tolist()
        if zero_impact_features:
            print(f"\n🚫 Found {len(zero_impact_features)} features with 0 total SHAP interaction contribution.")
            print("Suggested drop list:")
            for f in zero_impact_features:
                print(f" - {f}")
        else:
            print("\n✅ All features have some contribution.")

        # Optional: plot top and bottom
        fig, axs = plt.subplots(1, 2, figsize=(15, 5))
        feature_total_impact.tail(20).plot(kind="barh", ax=axs[0], title="Top 20 Contributors")
        feature_total_impact.head(20).plot(kind="barh", ax=axs[1], title="Bottom 20 Contributors")
        axs[0].grid(True)
        axs[1].grid(True)
        plt.tight_layout()
        plt.show()



    return preds


def run_regression_pipeline(
    lookahead: int,
    parquet_path: str,
    feature_groups: dict,
    model_list: list,
    train_end: str,
    val_end: str,
    splits: int = 3,
    trials: int = 20,
    study_folder: str = "optuna_studies"
):
    os.makedirs(study_folder, exist_ok=True)

    # === Load & split data ===
    df = pd.read_parquet(parquet_path)
    if df.index.name == 'datetime' or pd.api.types.is_datetime64_any_dtype(df.index):
        df = df.reset_index()
    if 'datetime' not in df.columns:
        raise KeyError("❌ 'datetime' column is missing.")
    
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('datetime')

    train = df[df['datetime'] < train_end]
    val = df[(df['datetime'] >= train_end) & (df['datetime'] < val_end)]
    test = df[df['datetime'] >= val_end]  # optional

    print(f"Train rows: {len(train)} | Val rows: {len(val)} | Test rows: {len(test)}")

    # === Features & labels ===
    X_train = train[feature_groups['tree']]
    X_val = val[feature_groups['tree']]

    reg_col = [col for col in df.columns if col.startswith("reg_target")][0]
    print(f"📌 Using regression target column: {reg_col}")
    y_train = train[reg_col]
    y_val = val[reg_col]

    results = {}

    # === Loop through models ===
    for model_name in model_list:
        print(f"\n=== 🔄 Running {model_name.upper()} training ===")
        study_name = f"{model_name}_reg_opt_{lookahead}"
        db_path = f"sqlite:///{study_folder}/{study_name}.db"

        if model_name == "xgboost":
            from xgboost import XGBRegressor
            def tuner(X, y): return tune_xgb_regressor(X, y, splits, trials, study_name, db_path)
            ModelClass = XGBRegressor

        elif model_name == "lightgbm":
            import lightgbm as lgb
            def tuner(X, y): return tune_lgb_regressor(X, y, splits, trials, study_name, db_path)
            ModelClass = lgb.LGBMRegressor

        elif model_name == "catboost":
            from catboost import CatBoostRegressor
            def tuner(X, y): return tune_catboost_regressor(X, y, splits, trials, study_name, db_path)
            ModelClass = CatBoostRegressor

        elif model_name == "rf":
            from sklearn.ensemble import RandomForestRegressor
            def tuner(X, y): return tune_rf_regressor(X, y, splits, trials, study_name, db_path)
            ModelClass = RandomForestRegressor

        elif model_name == "ridge":
            from sklearn.linear_model import Ridge
            def tuner(X, y): return tune_ridge_regressor(X, y, splits, trials, study_name, db_path)
            def ModelClass(**params): return make_pipeline(StandardScaler(), Ridge(**params))

        elif model_name == "elasticnet":
            from sklearn.linear_model import ElasticNet
            def tuner(X, y): return tune_elasticnet_regressor(X, y, splits, trials, study_name, db_path)
            def ModelClass(**params): return make_pipeline(StandardScaler(), ElasticNet(**params))

        else:
            print(f"⚠️ Unknown model: {model_name}")
            continue

        best_params = tuner(X_train, y_train)
        model = ModelClass(**best_params)
        model.fit(X_train, y_train)

        entry_prices = val['open'].shift(-1).reset_index(drop=True)
        exit_prices = val['close'].shift(-(1 + lookahead)).reset_index(drop=True)

        y_val_dates = val['datetime'].reset_index(drop=True)  # <- extract dates from validation set
        preds = analyze_regression(
            model_name, model, X_val, y_val,
            y_dates=y_val_dates,
            entry_prices=entry_prices,
            exit_prices=exit_prices
        )

        results[model_name] = {
            "model": model,
            "params": best_params,
            "predictions": preds
        }

    return results, df

In [ ]:
LOOKAHEAD=12
PARQUET_PATH = f"parquet/labeled_data_L12_PT2SL1VB12-model-research.parquet"
# Split date ranges
TRAIN_END = "2024-01-01"
VAL_END = "2025-01-01"

# Extract full list of columns from the DataFrame
df_sample = pd.read_parquet(PARQUET_PATH)

# Handle datetime index if needed
if df_sample.index.name == 'datetime' or df_sample.index.dtype.kind == 'M':
    df_sample = df_sample.reset_index()

# Define exclusion logic
non_feature_patterns = ['datetime', 'open', 'high', 'low', 'close', 'volume', 'target', 'label']
non_feature_cols = [col for col in df_sample.columns if any(pat in col.lower() for pat in non_feature_patterns)]

# Select only numeric, non-excluded features
tree_based_features = [
    col for col in df_sample.columns
    if col not in non_feature_cols and pd.api.types.is_numeric_dtype(df_sample[col])
]

FEATURE_GROUPS = {
    "tree": tree_based_features,
    # "linear": tree_based_features,
    # "sequential": tree_based_features
}

REGRESSION_MODEL_LIST = ["xgboost", "catboost", "lightgbm", "elasticnet"]

# Run regression pipeline
reg_results, df_all_reg = run_regression_pipeline(
    lookahead=LOOKAHEAD,
    parquet_path=PARQUET_PATH,
    feature_groups=FEATURE_GROUPS,
    model_list=REGRESSION_MODEL_LIST,
    train_end=TRAIN_END,
    val_end=VAL_END,
    splits=3,
    trials=10,
    study_folder="dbs"
)

MODEL_LIST = ["xgboost", "catboost", "rf", "lightgbm", "logreg", "ridge", "svc"]

# Run pipeline
results, label_encoder, df_all = run_classification_pipeline(
    lookahead=LOOKAHEAD,
    parquet_path=PARQUET_PATH,
    feature_groups=FEATURE_GROUPS,
    model_list=MODEL_LIST,
    train_end=TRAIN_END,
    val_end=VAL_END,
    splits=3,
    trials=10,
    num_classes=3,
    study_folder="dbs"
)



In [ ]:
# def run_lookahead_for_session_regression(LOOKAHEAD, cutoff, splits):
#     labeled = pd.read_parquet(f"labeled_data_{LOOKAHEAD}_session_less.parquet")

#     cutoff_date = pd.Timestamp(cutoff, tz="America/New_York")
#     train = labeled[labeled['datetime'] < cutoff_date]
#     test = labeled[labeled['datetime'] >= cutoff_date]

#     X_train_tree = train[tree_based_features]
#     X_test_tree = test[tree_based_features]

#     X_train_seq  = train[sequential_features]
#     X_test_seq = test[sequential_features]

#     y_train_tree = train['log_return']
#     y_test_tree = test['log_return']

#     y_train_seq = train['vol_adj_return']
#     y_test_seq = test['vol_adj_return']

#     y_train_transformed = np.sign(y_train_seq) * np.log1p(np.abs(y_train_seq))

#     print(f"Train range: {train['datetime'].min()} to {train['datetime'].max()} | Rows: {len(train)}")
#     print(f"Test range: {test['datetime'].min()} to {test['datetime'].max()} | Rows: {len(test)}")

#     ###########################
#     ########## Models #########
#     ###########################

#     def tune_xgboost(X_train, y_train):
#         def objective(trial):
#             params = {
#                 'n_estimators': 2000,
#                 'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.3, log=True),  # tighten low end
#                 'max_depth': trial.suggest_int('max_depth', 6, 14),  # more complex trees
#                 'subsample': trial.suggest_float('subsample', 0.7, 1.0),  # prevent underfitting
#                 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),  # prevent weak splits
#                 'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),  # reduce L1 regularization
#                 'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),  # reduce L2 regularization
#                 'min_child_weight': trial.suggest_int('min_child_weight', 1, 6),  # avoid pruning all splits
#                 'gamma': trial.suggest_float('gamma', 0.0, 1.0),  # allow moderate split pruning
#                 'tree_method': 'hist',
#             }

#             tscv = TimeSeriesSplit(n_splits=splits)
#             scores = []

#             for train_idx, val_idx in tscv.split(X_train):
#                 X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
#                 y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

#                 model = XGBRegressor(**params, random_state=42, eval_metric='rmse', early_stopping_rounds=20)
#                 model.fit(
#                     X_tr, y_tr,
#                     eval_set=[(X_val, y_val)],
#                     verbose=False
#                 )
#                 preds = model.predict(X_val)
#                 rmse = root_mean_squared_error(y_val, preds)
#                 scores.append(rmse)

#             return np.mean(scores)

#         study = optuna.create_study(
#             direction='minimize',
#             study_name=f'xgb_opt_reg_{LOOKAHEAD}',
#             sampler=optuna.samplers.TPESampler(seed=42),
#             pruner=optuna.pruners.SuccessiveHalvingPruner(min_resource=50, reduction_factor=4),
#             storage=f'sqlite:///xgb_opt_study_session_less.db',
#             load_if_exists=True
#         )
#         study.optimize(objective, n_trials=50)
#         return study.best_params

#     def tune_lightgbm(X_train, y_train):
#         def objective(trial):
#             params = {
#                 "n_estimators": trial.suggest_int("n_estimators", 300, 1500, step=100),
#                 "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.05, log=True),
#                 "max_depth": trial.suggest_int("max_depth", 3, 5),
#                 "num_leaves": trial.suggest_int("num_leaves", 10, 30),
#                 "min_child_samples": trial.suggest_int("min_child_samples", 10, 40),
#                 "subsample": trial.suggest_float("subsample", 0.6, 0.9),
#                 "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.9),
#                 "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0.05, 1.0),
#                 "reg_alpha": trial.suggest_float("reg_alpha", 1.0, 10.0),
#                 "reg_lambda": trial.suggest_float("reg_lambda", 1.0, 10.0),
#                 "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 0.2),
#                 "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 50, 200),
#                 "boosting_type": "gbdt",
#                 "verbosity": -1,
#                 "metric": "rmse"
#             }
#             tscv = TimeSeriesSplit(n_splits=splits)
#             scores = []
#             for train_idx, val_idx in tscv.split(X_train):
#                 X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
#                 y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

#                 model = LGBMRegressor(**params, random_state=42, n_jobs=-2)
#                 model.fit(
#                     X_tr, y_tr,
#                     eval_set=[(X_val, y_val)],
#                     eval_metric="rmse",
#                     callbacks=[early_stopping(stopping_rounds=50)]
#                 )
#                 preds = model.predict(X_val)
#                 rmse = root_mean_squared_error(y_val, preds)
#                 scores.append(rmse)
#                 print(f"Trial {trial.number} RMSE: {np.mean(scores):.6f} | Params: {params}")
#             return np.mean(scores)

#         study = optuna.create_study(
#             direction="minimize",
#             study_name=f"lgbm_opt_reg_{LOOKAHEAD}",
#             sampler=optuna.samplers.TPESampler(seed=42),
#             pruner=optuna.pruners.SuccessiveHalvingPruner(min_resource=50, reduction_factor=4),
#             storage=f"sqlite:///lgbm_opt_study_session_less.db",
#             load_if_exists=True
#         )
#         study.optimize(objective, n_trials=50)
#         return study.best_params

#     def tune_catboost(X_train, y_train):
#         def objective(trial):
#             params = {
#                 'iterations': 2000,
#                 'depth': trial.suggest_int('depth', 4, 8),
#                 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05, log=True),
#                 'loss_function': 'RMSE',
#                 'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 3.0, 10.0),
#                 'random_strength': trial.suggest_float('random_strength', 1.0, 5.0),
#                 'bootstrap_type': 'Bayesian',
#                 'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 1.0),
#                 'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
#             }

#             tscv = TimeSeriesSplit(n_splits=splits)
#             scores = []

#             for train_idx, val_idx in tscv.split(X_train):
#                 X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
#                 y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

#                 model = CatBoostRegressor(**params, random_state=42)
#                 model.fit(
#                     X_tr, y_tr,
#                     eval_set=(X_val, y_val),
#                     use_best_model=True,
#                     verbose=False,
#                     early_stopping_rounds=30
#                 )
#                 preds = model.predict(X_val)
#                 rmse = root_mean_squared_error(y_val, preds)
#                 scores.append(rmse)

#             return np.mean(scores)

#         study = optuna.create_study(
#             direction='minimize',
#             study_name=f'catboost_opt_reg_{LOOKAHEAD}',
#             sampler=optuna.samplers.TPESampler(seed=42),
#             pruner=optuna.pruners.SuccessiveHalvingPruner(min_resource=50, reduction_factor=4),
#             storage=f'sqlite:///catboost_opt_study_session_less.db',
#             load_if_exists=True
#         )
#         study.optimize(objective, n_trials=50)
#         return study.best_params

#     def tune_meta_xgb(X_meta, y_meta):
#         def objective(trial):
#             params = {
#                 'n_estimators': 1000,
#                 'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
#                 'max_depth': trial.suggest_int('max_depth', 2, 8),  # allow deeper if needed
#                 'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#                 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#                 'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 0.5),
#                 'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 0.5),
#                 'gamma': trial.suggest_float('gamma', 0.0, 1.0),  # extra regularization
#             }

#             tscv = TimeSeriesSplit(n_splits=splits)
#             scores = []

#             for train_idx, val_idx in tscv.split(X_meta):
#                 X_tr, X_val = X_meta.iloc[train_idx], X_meta.iloc[val_idx]
#                 y_tr, y_val = y_meta.iloc[train_idx], y_meta.iloc[val_idx]

#                 model = XGBRegressor(**params, random_state=42, n_jobs=-2, eval_metric='rmse')
#                 model.fit(
#                     X_tr, y_tr,
#                     eval_set=[(X_val, y_val)]
#                 )
#                 preds = model.predict(X_val)
#                 rmse = root_mean_squared_error(y_val, preds)
#                 scores.append(rmse)

#             return np.mean(scores)

#         study = optuna.create_study(
#             direction='minimize',
#             study_name=f'meta_xgb_reg_{LOOKAHEAD}',
#             sampler=optuna.samplers.TPESampler(seed=42),
#             pruner=optuna.pruners.MedianPruner(n_startup_trials=5),
#             storage=f'sqlite:///meta_xgb_session_less.db',
#             load_if_exists=True
#         )
#         study.optimize(objective, n_trials=50)
#         return study.best_params

#     def inverse_log_signed(x):
#         return np.sign(x) * (np.expm1(np.abs(x)))
#     ################################################
#     ####### Ensure index consistency
#     ####### Sequential #######
#     y_train_seq = y_train_seq.loc[X_train_seq.index]
#     y_test_seq = y_test_seq.loc[X_test_seq.index]

#     ################################################
#     ####### Tune models
#     ####### Tree Based #######
#     catboost_params     = tune_catboost(X_train_tree, y_train_transformed)
#     xgboost_params      = tune_xgboost(X_train_tree, y_train_transformed)
#    # lgbm_params         = tune_lightgbm(X_train_tree, y_train_transformed)
#     ####### Sequential #######
#     # N/A

#     ################################################
#     ####### Train models
#     ####### Tree Based #######
#     catboost    = CatBoostRegressor(**catboost_params, random_state=42, verbose=0)
#     xgboost     = XGBRegressor(**xgboost_params, random_state=42)
#    # lgbm        = LGBMRegressor(**lgbm_params, random_state=42)
#     catboost.fit(X_train_tree, y_train_transformed)
#     xgboost.fit(X_train_tree, y_train_transformed)
#   #  lgbm.fit(X_train_tree, y_train_transformed)
#     ####### Sequential #######
#     X_lstm = X_train_seq.values
#     y_lstm = y_train_transformed.values
#     lstm_model = LSTMWrapper(input_shape=X_lstm.shape[1])
#     lstm_model.fit(X_lstm, y_lstm)  # wrapper does the reshaping
#     X_lstm_test = X_test_seq.values
#     lstm_preds = lstm_model.predict(X_lstm_test)

#     X_cnn = X_train_seq.values.reshape((len(X_train_seq), X_train_seq.shape[1], 1))
#     y_cnn = y_train_seq.values
#     cnn_model = CNN1DWrapper(input_shape=(X_cnn.shape[1], 1))
#     cnn_model.fit(X_cnn, y_cnn)
#     X_cnn_test = X_test_seq.values.reshape((len(X_test_seq), X_test_seq.shape[1], 1))
#     cnn_preds = cnn_model.predict(X_cnn_test)

#     ################################################
#     ####### OOF Predicition
#     ####### Tree Based #######
#     oof_tree = generate_oof_predictions([catboost, xgboost], X_train_tree, y_train_transformed, splits)

#     print("\n🔍 Checking variance in OOF base model predictions Base model:")
#     print(oof_tree.describe())
#     print("Std per model:\n", oof_tree.std())
#     ####### Sequential #######
#     oof_preds_cnn = generate_oof_cnn(CNN1DWrapper, X_train_seq, y_train_seq, splits)

#     print("\n🔍 Checking variance in OOF base model predictions for CNN:")
#     print(pd.Series(oof_preds_cnn).describe())
#     print("Std:", np.std(oof_preds_cnn))

#     ################################################
#     ####### Meta Params and Training
#     ####### Tree Based #######
#     X_seq_np = X_train_seq.values
#     lstm_oof = generate_oof_lstm(LSTMWrapper, X_seq_np, y_train_transformed, splits)  # <- I can give you this

#     X_meta_train = pd.DataFrame({
#         'cat': oof_tree.iloc[:, 0],
#         'xgb': oof_tree.iloc[:, 1],
#         'lstm': lstm_oof
#     })

#     X_meta_train.describe()
#     print("Meta input std:\n", X_meta_train.std())
#     print("Meta input correlation:\n", X_meta_train.corr())

#     X_test_meta = pd.DataFrame({
#         'cat': catboost.predict(X_test_tree),
#         'xgb': xgboost.predict(X_test_tree),
#         'lstm': lstm_model.predict(X_test_seq.values)
#     })

#     meta_params = tune_meta_xgb(X_meta_train, y_train_transformed)
#     meta_model = XGBRegressor(**meta_params, random_state=42, objective='reg:squarederror', eval_metric='rmse')
#     meta_model.fit(X_meta_train, y_train_transformed)

#     ################################################
#     ####### Evaluate Model
#     def evaluate_model(name, model, Xtr, Xte, ytr, yte, transformed=False):
#         train_preds = model.predict(Xtr)
#         test_preds = model.predict(Xte)

#         if transformed:
#         # Inverse-transform predictions
#             train_preds = np.sign(train_preds) * (np.expm1(np.abs(train_preds)))
#             test_preds = np.sign(test_preds) * (np.expm1(np.abs(test_preds)))
#             ytr = np.sign(ytr) * (np.expm1(np.abs(ytr)))
        
#         train_mse = mean_squared_error(ytr, train_preds)
#         test_mse = mean_squared_error(yte, test_preds)
#         overfit_ratio = test_mse / train_mse if train_mse != 0 else float('inf')

#         print(f"\n📊 {name} Performance:")
#         print(f"Train MSE: {train_mse:.8f}")
#         print(f"Test MSE: {test_mse:.8f}")
#         print(f"Overfit ratio (Test / Train): {overfit_ratio:.2f}")
#         if overfit_ratio > 1.5:
#             print("⚠️ Potential overfitting detected.")
#         elif overfit_ratio < 0.7:
#             print("⚠️ Possibly underfitting.")
#         else:
#             print("✅ Generalization looks reasonable.")
#         return test_preds
    
#     ####### Tree Based #######
#     print("\nEvaluation XGBoost")
#     preds_xgboost   = evaluate_model("XGBoostRegressor", xgboost, X_train_tree, X_test_tree, y_train_transformed, y_test_seq, transformed=True)
#     print("\nEvaluation CatBoost")
#     preds_catboost  = evaluate_model("CatBoostRegressor", catboost, X_train_tree, X_test_tree, y_train_transformed, y_test_seq, transformed=True)
#     print("\nEvaluation Stack")
#     preds_stack     = evaluate_model("StackingRegressor", meta_model, X_meta_train, X_test_meta, y_train_transformed.values, y_test_seq.values, transformed=True)
#  #   print("\nEvaluation LGBM")
# #    preds_lgbm      = evaluate_model("LightGBM", lgbm, X_train_tree, X_test_tree, y_train_transformed, y_test_tree, transformed=True)
#     ####### Sequential #######
#     X_cnn_train = X_train_seq.values.reshape((len(X_train_seq), X_train_seq.shape[1], 1))
#     X_cnn_test = X_test_seq.values.reshape((len(X_test_seq), X_test_seq.shape[1], 1))

#     print("\nEvaluation LSTM")
#     preds_lstm       = evaluate_model("LSTM", lstm_model, X_train_seq.values, X_test_seq.values, y_train_transformed.values, y_test_seq.values, transformed=True)
#     print("\nEvaluation CNN")
#     preds_cnn      = evaluate_model("CNN", cnn_model, X_cnn_train, X_cnn_test, y_train_seq.values, y_test_seq.values)

#     ################################################
#     ####### Target Distribution
#     ####### Tree based #######
#     print("\n🔍 Target distribution Tree:")
#     print(y_train_tree.describe())
#     ####### Sequential #######
#     print("\n🔍 Target distribution Seq:")
#     print(y_train_seq.describe())
    
#     ################################################
#     ####### Choose final model
#     ####### Tree Based #######
#     # print("\n🔍 Checking prediction variance from LGBM model:")
#     # print(f"Min: {preds_lgbm.min():.8f}")
#     # print(f"Max: {preds_lgbm.max():.8f}")
#     # print(f"Mean: {preds_lgbm.mean():.8f}")
#     # print(f"Std Dev: {preds_lgbm.std():.8f}")
#     # print(f"First 5 Predictions: {preds_lgbm[:5]}")

#     # mae_lgbm = mean_absolute_error(y_test_tree, preds_lgbm)
#     # rmse_lgbm = np.sqrt(mean_squared_error(y_test_tree, preds_lgbm))
#     # r2_lgbm = r2_score(y_test_tree, preds_lgbm)

#     # print(f"MAE: {mae_lgbm:.4f}")
#     # print(f"RMSE: {rmse_lgbm:.4f}")
#     # print(f"R²: {r2_lgbm:.4f}")
#     ####### Stacked Model #######
#     print("\n🔍 Checking prediction variance from Stack model:")
#     print(f"Min: {preds_stack.min():.8f}")
#     print(f"Max: {preds_stack.max():.8f}")
#     print(f"Mean: {preds_stack.mean():.8f}")
#     print(f"Std Dev: {preds_stack.std():.8f}")
#     print(f"First 5 Predictions: {preds_stack[:5]}")

#     mae = mean_absolute_error(y_test_seq, preds_stack)
#     rmse = np.sqrt(mean_squared_error(y_test_seq, preds_stack))
#     r2 = r2_score(y_test_seq, preds_stack)

#     print(f"MAE: {mae:.4f}")
#     print(f"RMSE: {rmse:.4f}")
#     print(f"R²: {r2:.4f}")
#     ####### Sequential Solo #######
#     print("\n🔍 Checking prediction variance from CNN model:")
#     print(f"Min: {preds_cnn.min():.8f}")
#     print(f"Max: {preds_cnn.max():.8f}")
#     print(f"Mean: {preds_cnn.mean():.8f}")
#     print(f"Std Dev: {preds_cnn.std():.8f}")
#     print(f"First 5 Predictions: {preds_cnn[:5]}")

#     mae_cnn = mean_absolute_error(y_test_seq, preds_cnn)
#     rmse_cnn = np.sqrt(mean_squared_error(y_test_seq, preds_cnn))
#     r2_cnn = r2_score(y_test_seq, preds_cnn)

#     print(f"MAE: {mae_cnn:.4f}")
#     print(f"RMSE: {rmse_cnn:.4f}")
#     print(f"R²: {r2_cnn:.4f}")

#     metadata = {
#         "lookahead": LOOKAHEAD,
#         "xgboost_params": xgboost_params,
#         "catboost_params": catboost_params,
#         "meta_params": meta_params,
#         # "lgbm_params": lgbm_params
#     }
#     with open(f"regression_metadata_{LOOKAHEAD}.json", "w") as f:
#         json.dump(metadata, f, indent=2)
        
#     joblib.dump(meta_model, f"stack_model_regression_LOOKAHEAD_{LOOKAHEAD}_session_less.pkl")
#     joblib.dump(cnn_model, f"cnn_model_regression_LOOKAHEAD_{LOOKAHEAD}_session_less.pkl")
#    #joblib.dump(lgbm, f"lgbm_model_regression_LOOKAHEAD_{LOOKAHEAD}_session_less.pkl")

#     return {
#         'lookahead': LOOKAHEAD,
#         'preds_stack': preds_stack,
#         'preds_cnn': preds_cnn,
#      #   'preds_lgbm': preds_lgbm,
#         'X_test_seq': X_test_seq,
#         'X_test_meta': X_test_meta,
#         'true_values': y_test_seq.values
#     }

In [ ]:
# def run_lookahead_for_session_classification(LOOKAHEAD, cutoff, splits):
#     labeled = pd.read_parquet(f"labeled_data_{LOOKAHEAD}_session_less.parquet")

#     cutoff_date = pd.Timestamp(cutoff, tz="America/New_York")
#     train = labeled[labeled['datetime'] < cutoff_date]
#     test = labeled[labeled['datetime'] >= cutoff_date]

#     X_train_tree = train[tree_based_features]
#     X_test_tree = test[tree_based_features]
#     X_train_seq  = train[sequential_features]
#     X_test_seq = test[sequential_features]
#     X_train_linear  = train[linear_features]
#     X_test_linear = test[linear_features]

#     le = LabelEncoder()
#     y_train_class = pd.Series(le.fit_transform(train['triple_barrier_label']), index=train.index)
#     y_test_class = pd.Series(le.transform(test['triple_barrier_label']), index=test.index)

#     X_train_linear = X_train_linear.loc[y_train_class.index]
#     X_test_linear = X_test_linear.loc[y_test_class.index]

#     print(f"Train range: {train['datetime'].min()} to {train['datetime'].max()} | Rows: {len(train)}")
#     print(f"Test range: {test['datetime'].min()} to {test['datetime'].max()} | Rows: {len(test)}")

#     ###########################
#     ########## Models #########
#     ###########################

#     def tune_xgboost(X_train, y_train):
#         def objective(trial):

#             params = {
#                 'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
#                 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
#                 'max_depth': trial.suggest_int('max_depth', 3, 12),
#                 'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#                 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#                 'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
#                 'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
#                 'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#                 'gamma': trial.suggest_float('gamma', 0.0, 5.0),
#                 'eval_metric': 'logloss',
#                 'objective': 'multi:softmax',  # or 'multi:softprob' if you need probabilities
#                 'num_class': len(np.unique(y_train))
#             }

#             tscv = TimeSeriesSplit(n_splits=splits)
#             scores = []

#             for train_idx, val_idx in tscv.split(X_train):
#                 X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
#                 y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

#                 sample_weights = compute_sample_weight(class_weight='balanced', y=y_tr)

#                 model = XGBClassifier(**params, random_state=42, n_jobs=-1)
#                 model.fit(X_tr, y_tr, sample_weight=sample_weights)

#                 preds = model.predict(X_val)
#                 score = f1_score(y_val, preds, average='macro')
#                 scores.append(score)

#             print(f"Trial {trial.number} F1 Score: {np.mean(scores):.5f} | Params: {params}")
#             return np.mean(scores)

#         study = optuna.create_study(
#             direction='maximize',
#             study_name=f'xgb_opt_class_{LOOKAHEAD}',
#             sampler=optuna.samplers.TPESampler(seed=42),
#             pruner=optuna.pruners.MedianPruner(n_startup_trials=5),
#             storage=f'sqlite:///xgb_opt_study_session_less.db',
#             load_if_exists=True
#         )
#         study.optimize(objective, n_trials=2)
#         return study.best_params

#     def tune_rf(X_train, y_train):
#         def objective(trial):
#             params = {
#                 "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
#                 "max_depth": trial.suggest_int("max_depth", 3, 20),
#                 "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
#                 "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
#                 "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
#                 "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
#                 "class_weight": trial.suggest_categorical("class_weight", [None, "balanced", "balanced_subsample"]),
#                 "criterion": trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"]),
#             }

#             tscv = TimeSeriesSplit(n_splits=splits)
#             model = RandomForestClassifier(**params, random_state=42, n_jobs=-1)

#             scores = cross_val_score(model, X_train, y_train, cv=tscv, scoring="f1_macro", n_jobs=-1)
#             print(f"Trial {trial.number} F1 Score: {scores.mean():.5f} | Params: {params}")
#             return scores.mean()

#         study = optuna.create_study(
#             direction="maximize",
#             study_name=f"rf_opt_class_{LOOKAHEAD}",
#             sampler=optuna.samplers.TPESampler(seed=42),
#             pruner=optuna.pruners.MedianPruner(n_startup_trials=5),
#             storage=f"sqlite:///rf_opt_study_session_less.db",
#             load_if_exists=True
#         )
#         study.optimize(objective, n_trials=2)
#         return study.best_params

#     def tune_catboost(X_train, y_train):
#         def objective(trial):
#             bootstrap_type = trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli'])

#             class_weights = compute_class_weight(
#                 class_weight='balanced',
#                 classes=np.unique(y_train),
#                 y=y_train
#             )

#             params = {
#                 'iterations': trial.suggest_int('iterations', 300, 1500, step=100),
#                 'depth': trial.suggest_int('depth', 4, 10),
#                 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
#                 'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
#                 'random_strength': trial.suggest_float('random_strength', 0.5, 5.0),
#                 'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
#                 'bootstrap_type': bootstrap_type,
#                 'loss_function': 'MultiClass',
#                 'eval_metric': 'TotalF1',
#                 'class_weights': class_weights.tolist(),
#                 'verbose': 0
#             }

#             if bootstrap_type == 'Bayesian':
#                 params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0.0, 1.0)

#             model = CatBoostClassifier(**params, random_state=42)

#             tscv = TimeSeriesSplit(n_splits=splits)
#             scores = cross_val_score(model, X_train, y_train, cv=tscv, scoring='f1_macro', n_jobs=-1)
#             print(f"Trial {trial.number} F1 Score: {scores.mean():.5f} | Params: {params}")
#             return scores.mean()

#         study = optuna.create_study(
#             direction='maximize',
#             study_name=f'catboost_opt_class_{LOOKAHEAD}',
#             sampler=optuna.samplers.TPESampler(seed=42),
#             pruner=optuna.pruners.MedianPruner(n_startup_trials=5),
#             storage=f'sqlite:///catboost_opt_study_session_less.db',
#             load_if_exists=True
#         )
#         study.optimize(objective, n_trials=2)
#         return study.best_params

#     def tune_meta_logreg(X_meta, y_meta):
#         def objective(trial):
#             penalty = trial.suggest_categorical("penalty", ["l2", None])
#             if penalty is not None:
#                 C = trial.suggest_float("C", 1e-4, 10.0, log=True)
#             else:
#                 C = 1.0  # default, unused

#             class_weight = trial.suggest_categorical("class_weight", [None, "balanced"])

#             params = {
#                 "penalty": penalty,
#                 "C": C,
#                 "solver": "lbfgs",
#                 "max_iter": 2000,
#                 "class_weight": class_weight,
#             }

#             model = make_pipeline(StandardScaler(), LogisticRegression(**params, random_state=42))
#             tscv = TimeSeriesSplit(n_splits=splits)

#             scores = cross_val_score(model, X_meta, y_meta, cv=tscv, scoring="f1_macro", n_jobs=-1)
#             print(f"Trial {trial.number} F1 Score: {scores.mean():.5f} | Params: {params}")
#             return scores.mean()

#         study = optuna.create_study(
#             direction="maximize",
#             study_name=f"meta_logreg_class_{LOOKAHEAD}",
#             sampler=optuna.samplers.TPESampler(seed=42),
#             pruner=optuna.pruners.MedianPruner(n_startup_trials=5),
#             storage=f"sqlite:///meta_logreg_stack_session_less.db",
#             load_if_exists=True
#         )
#         study.optimize(objective, n_trials=2)  # adjust trial count as needed
#         return study.best_params

#     def tune_lstm_classifier_with_optuna(X, y, splits, lookahead, num_classes=2):
#         def objective(trial):
#             units = trial.suggest_int("units", 16, 128, step=16)
#             lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
#             batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
#             epochs = trial.suggest_int("epochs", 5, 30)

#             scores = []
#             tscv = TimeSeriesSplit(n_splits=splits)

#             for train_idx, val_idx in tscv.split(X):
#                 X_tr, X_val = X[train_idx], X[val_idx]
#                 y_tr, y_val = y[train_idx], y[val_idx]

#                 model = LSTMClassifierWrapper(
#                     input_shape=X.shape[1],
#                     units=units,
#                     lr=lr,
#                     epochs=epochs,
#                     batch_size=batch_size,
#                     verbose=0,
#                     num_classes=num_classes
#                 )
#                 model.fit(X_tr, y_tr)
#                 preds = model.predict(X_val)
#                 acc = accuracy_score(y_val, preds)
#                 scores.append(acc)

#             mean_acc = np.mean(scores)
#             print(f"Trial {trial.number} Accuracy: {mean_acc:.5f} | Params: units={units}, lr={lr}, batch={batch_size}, epochs={epochs}")
#             return mean_acc

#         study = optuna.create_study(
#             direction="maximize",
#             study_name="lstm_class_opt",
#             storage=f"sqlite:///lstm_class_opt_study{lookahead}_session_less.db",
#             load_if_exists=True
#         )
#         study.optimize(objective, n_trials=2)
#         print("Best trial:", study.best_trial.params)
#         return study.best_trial.params
#     ################################################
#     ####### Ensure index consistency
#     ####### Sequential #######
#     y_train_seq = y_train_class.loc[X_train_seq.index]
#     y_test_seq = y_test_class.loc[X_test_seq.index]

#     ################################################
#     ####### Tune models
#     ####### Tree Based #######
#     catboost_params     = tune_catboost(X_train_tree, y_train_class)
#     xgboost_params      = tune_xgboost(X_train_tree, y_train_class)
#     rf_params         = tune_rf(X_train_tree, y_train_class)
#     ####### Sequential #######
#     X_lstm = X_train_seq.values
#     y_lstm = y_train_class.values

#     ################################################
#     ####### Train models
#     ####### Tree Based #######
#     xgb_weights = compute_sample_weight('balanced', y_train_class)
#     cb_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_class), y=y_train_class)
#     catboost    = CatBoostClassifier(**catboost_params, random_state=42, class_weights=cb_weights.tolist(), verbose=0)
#     xgboost     = XGBClassifier(**xgboost_params, random_state=42)
#     rf          = RandomForestClassifier(**rf_params, random_state=42)
#     catboost.fit(X_train_tree, y_train_class)
#     xgboost.fit(X_train_tree, y_train_class, sample_weight=xgb_weights)
#     rf.fit(X_train_tree, y_train_class)
#     ####### Sequential #######
#     lstm_model = LSTMClassifierWrapper(input_shape=X_lstm.shape[1])
#     lstm_model.fit(X_lstm, y_lstm)  # wrapper does the reshaping
#     X_lstm_test = X_test_seq.values
#     lstm_preds = lstm_model.predict(X_lstm_test)
#     ################################################
#     ####### OOF Predicition
#     ####### Tree Based #######
#     oof_tree = generate_oof_predictions_class([catboost, rf], X_train_tree, y_train_class, splits)
#     oof_lstm = generate_oof_lstm_classifier(LSTMClassifierWrapper, X_lstm, y_lstm, splits)  # <- Uses sequential input

#     ################################################
#     ####### Train Meta Model
#     ####### Tree Based #######
#     X_meta_train = pd.DataFrame({
#         'cat': oof_tree.iloc[:, 0].values,
#         'rf': oof_tree.iloc[:, 1].values,
#         'lstm': oof_lstm.values
#     }, index=y_train_class.index)

#     X_meta_test = pd.DataFrame({
#         "cat": catboost.predict(X_test_tree).flatten(),
#         "rf": rf.predict(X_test_tree).flatten(),
#         "lstm": lstm_model.predict(X_test_seq.values).flatten()
#     })

#     X_meta_train_combined = pd.concat([
#         X_meta_train.reset_index(drop=True),
#         X_train_linear.reset_index(drop=True)
#     ], axis=1)

#     X_meta_test_combined = pd.concat([
#         X_meta_test.reset_index(drop=True),
#         X_test_linear.reset_index(drop=True)
#     ], axis=1)

#     meta_params = tune_meta_logreg(X_meta_train_combined, y_train_class)
#     meta_model = make_pipeline(StandardScaler(),LogisticRegression(**meta_params, random_state=42))
#     meta_model.fit(X_meta_train_combined, y_train_class)

#     ################################################
#     ####### Evaluate Model
#     def evaluate_model(name, model, Xtr, Xte, ytr, yte):
#         train_preds = model.predict(Xtr)
#         test_preds = model.predict(Xte)

#         train_acc = accuracy_score(ytr, train_preds)
#         test_acc = accuracy_score(yte, test_preds)

#         print(f"\n📊 {name} Classification Accuracy:")
#         print(f"Train Accuracy: {train_acc:.4f}")
#         print(f"Test Accuracy: {test_acc:.4f}")

#         return test_preds
    
#     ####### Tree Based #######
#     print("\nXGBoost")
#     preds_xgboost   = evaluate_model("XGBoostRegressor", xgboost, X_train_tree, X_test_tree, y_train_seq, y_test_seq)
#     print("\nCatboost")
#     preds_catboost  = evaluate_model("CatBoostRegressor", catboost, X_train_tree, X_test_tree, y_train_seq, y_test_seq)
#     print("\nRF")
#     preds_rf        = evaluate_model("RandomForest", rf, X_train_tree, X_test_tree, y_train_seq, y_test_seq)
#     print("\nLSTM")
#     preds_lstm       = evaluate_model("LSTM", lstm_model, X_train_seq.values, X_test_seq.values, y_train_seq.values, y_test_seq.values)
#     print("\nMeta Model")
#     preds_stack     = evaluate_model("StackingRegressor", meta_model, X_meta_train_combined, X_meta_test_combined, y_train_class.values, y_test_class.values)

#     ################################################
#     ####### Target Distribution
#     print("\n🔍 Target distribution:")
#     print(y_train_class.describe())
    
#     ################################################
#     ####### Choose final model
#     ####### Tree Based #######
#     preds_xgboost = xgboost.predict(X_test_tree)
#     print("\n🔍 Checking XGBoost prediction distribution (classification):")
#     print(f"Classes predicted: {np.unique(preds_xgboost)}")
#     print(f"Prediction counts:\n{pd.Series(preds_xgboost).value_counts()}")

#     # Classification metrics
#     acc = accuracy_score(y_test_class, preds_xgboost)
#     f1 = f1_score(y_test_class, preds_xgboost, average='macro')
#     print(f"Accuracy: {acc:.4f}")
#     print(f"F1 Score (macro): {f1:.4f}")
#     print("\nClassification report:")
#     print(classification_report(y_test_class, preds_xgboost))

#     ####### Stacked Model #######
#     preds_meta_model = meta_model.predict(X_meta_test_combined)
#     print("\n🔍 Checking Meta Model prediction distribution (classification):")
#     print(f"Classes predicted: {np.unique(preds_meta_model)}")
#     print(f"Prediction counts:\n{pd.Series(preds_meta_model).value_counts()}")

#     # Classification metrics
#     acc = accuracy_score(y_test_class, preds_meta_model)
#     f1 = f1_score(y_test_class, preds_meta_model, average='macro')
#     print(f"Accuracy: {acc:.4f}")
#     print(f"F1 Score (macro): {f1:.4f}")
#     print("\nClassification report:")
#     print(classification_report(y_test_class, preds_meta_model))

#     metadata = {
#         "lookahead": LOOKAHEAD,
#         "xgboost_params": xgboost_params,
#         "catboost_params": catboost_params,
#         "rf_params": rf_params,
#         "meta_params": meta_params,
#     }
#     with open(f"classifier_metadata_{LOOKAHEAD}.json", "w") as f:
#         json.dump(metadata, f, indent=2)
        
#     joblib.dump(meta_model, f"stack_model_classifier_LOOKAHEAD_{LOOKAHEAD}_session_less.pkl")
#     joblib.dump(xgboost, f"xgboost_model_classifier_LOOKAHEAD_{LOOKAHEAD}_session_less.pkl")

#     return {
#         'lookahead': LOOKAHEAD,
#         'preds_stack': meta_model.predict_proba(X_meta_test_combined),
#         'preds_xgboost': xgboost.predict_proba(X_test_tree),
#         'X_test_tree': X_test_tree,
#         'X_test_linear': X_test_linear,
#         'X_meta_test_combined': X_meta_test_combined,
#         'true_values': y_test_class.values,
#         'label_encoder': le
#     }

##### Running Train

In [ ]:
# Regression Training
lookahead_values = [5, 10, 20]
reg_results = []

for val in lookahead_values:
    cutoff = "2025-04-01"
    splits=4
    regression_models = run_lookahead_for_session_regression(val, cutoff, splits)
    reg_results.append(regression_models)

In [ ]:
# Classification Training
lookahead_values = [10]
class_results = []

for val in lookahead_values:
    cutoff = "2025-04-01"
    splits=4
    classification_models = run_lookahead_for_session_classification(val, cutoff, splits)
    class_results.append(classification_models)

# Backtesting

##### Regression StandAlone Backtesting

In [ ]:
all_results = []
thresholds = [0.000002]

for result in reg_results:
    lookahead = result['lookahead']
    preds_stack = result['preds_stack']  # or 'preds_cnn'
    preds_cnn = result['preds_cnn']
    X_test_combined = result['X_test_meta']  # or 'X_test_seq'
    y_test = result['true_values']
    labeled = pd.read_parquet(f"labeled_data_{lookahead}_session_less.parquet")
    df_backtest = labeled.copy()

    print(f"\n🔎 Predicted return range for LOOKAHEAD={lookahead}: STACK: min={preds_stack.min():.8f}, max={preds_stack.max():.8f} | CNN: min={preds_cnn.min():.8f}, max={preds_cnn.max():.8f}")
    for params in combinations:
        for thresh in thresholds:
            results = evaluate_regression(
                X_test=X_test_combined,
                preds_stack=preds_stack,
                preds_cnn=preds_cnn,
                labeled=labeled,
                df=df_backtest,
                avoid_funcs=avoid_funcs,
                SL_ATR_MULT=params['SL_ATR_MULT'],
                TP_ATR_MULT=params['TP_ATR_MULT'],
                TRAIL_START_MULT=params['TRAIL_START_MULT'],
                TRAIL_STOP_MULT=params['TRAIL_STOP_MULT'],
                TICK_VALUE=params['TICK_VALUE'],
                is_same_session=is_same_session,
                long_thresh=thresh,
                short_thresh=-thresh,
                base_contracts=1,
                max_contracts=5,
                skip_weak_conf=True,
                weak_conf_zscore=0.2
            )

            results['params'] = params
            results['threshold'] = thresh
            all_results.append(results)

            print(f"\n\n🔍 Evaluating with params: {params}")

            print(
                f"\n✅ LOOKAHEAD={lookahead} | Threshold={thresh}"
                f"\nPnL: ${results['pnl']:.2f}"
                f"\nTrades: {results['trades']}"
                f"\nWin Rate: {results['win_rate']:.2%}"
                f"\nExpectancy: {results['expectancy']:.2f}"
                f"\nProfit Factor: {results['profit_factor']:.2f}"
                f"\nSharpe Ratio: {results['sharpe']:.2f}"
                f"\nLong Trades: {results['long_trades']} | Short Trades: {results['short_trades']}"
            )

            print("Avoid Hits:")
            for name, count in results['avoid_hits'].items():
                print(f" - {name}: {count}")

            if not results['results'].empty and 'pnl' in results['results'].columns:
                print("\n🔢 Top 5 PnL trades:")
                print(results['results'].sort_values(by='pnl', ascending=False).head(5))

                print("\n🔻 Bottom 5 PnL trades:")
                print(results['results'].sort_values(by='pnl', ascending=True).head(5))
            else:
                print("\n⚠️ No trades executed, skipping PnL trade breakdown.")


summary_df = pd.DataFrame([{
    'pnl': r['pnl'],
    'sharpe': r['sharpe'],
    'expectancy': r['expectancy'],
    'profit_factor': r['profit_factor'],
    'win_rate': r['win_rate'],
    'trades': r['trades'],
    **r['params']
} for r in all_results])
top = summary_df.sort_values(by='sharpe', ascending=False).head(10)
print("\n🏁 Top 10 Configurations Across All Lookaheads:")
print(top)

##### Classification StandAlone Backtesting

In [ ]:
all_classification_results = []

for result in class_results:
    lookahead = result['lookahead']
    preds_stack = result['preds_stack']  # or 'preds_cnn'
    preds_xgboost = result['preds_xgboost']
    X_test_combined = result['X_meta_test_combined']  # or 'X_test_seq'
    y_test = result['true_values']
    labeled = pd.read_parquet(f"labeled_data_{lookahead}_session_less.parquet")
    df_backtest = labeled.copy()
    le = result['label_encoder']

    for params in combinations:
        # X_test, preds_stack, preds_cnn, preds_lgbm, 
        results = evaluate_classification(
            X_test=X_test_combined,
            preds_stack=preds_stack,
            preds_xgboost=preds_xgboost,
            labeled=labeled,
            df=df_backtest,
            avoid_funcs=avoid_funcs,
            le=le,
            SL_ATR_MULT=params['SL_ATR_MULT'],
            TP_ATR_MULT=params['TP_ATR_MULT'],
            TRAIL_START_MULT=params['TRAIL_START_MULT'],
            TRAIL_STOP_MULT=params['TRAIL_STOP_MULT'],
            TICK_VALUE=params['TICK_VALUE'],
            is_same_session=is_same_session,
            base_contracts=1,
            max_contracts=5,
            skip_weak_conf=True,
            weak_conf_zscore=0.2
        )

        results['params'] = params
        all_classification_results.append(results)

        print(f"\n\n🔍 Evaluating with params: {params}")

        print(
            f"\n✅ LOOKAHEAD={lookahead} | Threshold={thresh}"
            f"\nPnL: ${results['pnl']:.2f}"
            f"\nTrades: {results['trades']}"
            f"\nWin Rate: {results['win_rate']:.2%}"
            f"\nExpectancy: {results['expectancy']:.2f}"
            f"\nProfit Factor: {results['profit_factor']:.2f}"
            f"\nSharpe Ratio: {results['sharpe']:.2f}"
            f"\nLong Trades: {results['long_trades']} | Short Trades: {results['short_trades']}"
        )

        print("Avoid Hits:")
        for name, count in results['avoid_hits'].items():
            print(f" - {name}: {count}")

        if not results['results'].empty and 'pnl' in results['results'].columns:
            print("\n🔢 Top 5 PnL trades:")
            print(results['results'].sort_values(by='pnl', ascending=False).head(5))

            print("\n🔻 Bottom 5 PnL trades:")
            print(results['results'].sort_values(by='pnl', ascending=True).head(5))
        else:
            print("\n⚠️ No trades executed, skipping PnL trade breakdown.")


summary_df = pd.DataFrame([{
    'pnl': r['pnl'],
    'sharpe': r['sharpe'],
    'expectancy': r['expectancy'],
    'profit_factor': r['profit_factor'],
    'win_rate': r['win_rate'],
    'trades': r['trades'],
    **r['params']
} for r in all_results])
top = summary_df.sort_values(by='sharpe', ascending=False).head(10)
print("\n🏁 Top 10 Configurations Across All Lookaheads:")
print(top)

##### Combo Backtesting

In [ ]:
all_combo_results = []
thresholds = [0.0000035]

for reg_result, class_result in zip(reg_results, class_results):
    lookahead = reg_result['lookahead']
    preds_stack_reg = reg_result['preds_stack']
    preds_cnn_reg = reg_result['preds_cnn']
    X_test_combo = reg_result['X_test_meta']
    y_test = reg_result['true_values']

    preds_stack_class = class_result['preds_stack']  # shape (n_samples, n_classes)
    preds_xgboost_class = class_result['preds_xgboost']
    le = class_result['label_encoder']
    labeled = pd.read_parquet(f"labeled_data_{lookahead}_session_less.parquet")
    df_backtest = labeled.copy()

    print(f"\n🔎 LOOKAHEAD={lookahead} Regression Prediction Range:")
    print(f"Stack Min: {np.min(preds_stack_reg):.6f} | Max: {np.max(preds_stack_reg):.6f}")
    print(f"CNN   Min: {np.min(preds_cnn_reg):.6f} | Max: {np.max(preds_cnn_reg):.6f}")
    print(f"Classification Unique Labels: {np.unique(le.inverse_transform(np.argmax(preds_stack_class, axis=1)))}")

    # ✅ Align classifier and regression predictions to common length
    min_len = min(len(X_test_combo), len(preds_stack_class), len(preds_xgboost_class), len(preds_stack_reg), len(preds_cnn_reg))
    X_test_trimmed = X_test_combo.iloc[:min_len]
    probs_stack_aligned = np.array(preds_stack_class)[:min_len]
    probs_xgb_aligned = np.array(preds_xgboost_class)[:min_len]
    preds_stack_reg = np.array(preds_stack_reg)[:min_len]
    preds_cnn_reg = np.array(preds_cnn_reg)[:min_len]

    for params in combinations:
        for thresh in thresholds:
            results = evaluate_combo(
                X_test=X_test_trimmed,
                preds_reg_stack=preds_stack_reg,
                preds_reg_cnn=preds_cnn_reg,
                le=le,
                probs_class_stack=probs_stack_aligned,
                probs_class_xgboost=probs_xgb_aligned,
                labeled=labeled,
                df=df_backtest,
                avoid_funcs=avoid_funcs,
                SL_ATR_MULT=params['SL_ATR_MULT'],
                TP_ATR_MULT=params['TP_ATR_MULT'],
                TRAIL_START_MULT=params['TRAIL_START_MULT'],
                TRAIL_STOP_MULT=params['TRAIL_STOP_MULT'],
                TICK_VALUE=params['TICK_VALUE'],
                is_same_session=is_same_session,
                long_threshold=thresh,
                short_threshold=-thresh,
                base_contracts=1,
                max_contracts=5,
                skip_weak_conf=True,
                weak_conf_zscore=0.2
            )

            results['params'] = params
            results['threshold'] = thresh
            results['lookahead'] = lookahead
            all_combo_results.append(results)

            print(f"\n✅ LOOKAHEAD={lookahead} | Threshold={thresh}")
            print(f"PnL: ${results['pnl']:.2f}")
            print(f"Trades: {results['trades']}")
            print(f"Win Rate: {results['win_rate']:.2%}")
            print(f"Expectancy: {results['expectancy']:.2f}")
            print(f"Profit Factor: {results['profit_factor']:.2f}")
            print(f"Sharpe Ratio: {results['sharpe']:.2f}")
            print(f"Long Trades: {results['long_trades']} | Short Trades: {results['short_trades']}")

            print("Avoid Hits:")
            for name, count in results['avoid_hits'].items():
                print(f" - {name}: {count}")

            if not results['results'].empty and 'pnl' in results['results'].columns:
                print("\n🔢 Top 5 PnL trades:")
                print(results['results'].sort_values(by='pnl', ascending=False).head(5))

                print("\n🔻 Bottom 5 PnL trades:")
                print(results['results'].sort_values(by='pnl', ascending=True).head(5))
            else:
                print("\n⚠️ No trades executed, skipping PnL trade breakdown.")
